# Rewriting Data Loader

In [2]:
import pickle
import pandas as pd
import numpy as np

In [3]:
from lightgbm import LGBMRegressor

I wanted to use vaex because I think it's faster - but I encountered a lot of problems with it not having all the features of pandas. So I'm switching back to pandas, unfortunately.

## Train Features

Actually, there are no train features - this is where the target is stored. At test time, targets are only revealed 2 days after the current date, so train will probably have to be 2 days behind current date. I'll come back to handle this later.

In [4]:
import pandas as pd
import datetime as dt
import numpy as np

import pandas as pd
import datetime as dt
import numpy as np
import time


class TrainLoader:
    """Stores the data in the 'train' file or 'test' files (which typically consists of target and grouping variables)"""
    def __init__(self, df):
        self.df = df
        self.recent_excerpt = self.init_recent_excerpt(df)

    def init_recent_excerpt(self, df):
        """Gets only the most recent data from the df, to reduce processing time for future feature creation"""
        pass

    def add_test_df(self, df):
        """adds the test df to the data"""
        pass  

## Revealed Targets Features

We'll be joining with the train_df on data_block_id, prediction_unit_id, is_consumption and hour.

At test time, we need to predict a day's worth of energy production/consumption, and we'll get a single datablock's worth of revealed info for our day. Each data_block_id gives us a single day of data 2 days in the past. In previous features, I moved the date 2 days in the future and joined on datetime. But now that I'm doing it from scratch, why not reimagine it? Seeing as the data is 2 days behind, there is no reason to stick to only the equivalent hour of the day and before it.

Note: because revealed targets are 2 days behind, the weekly period is offset by two days I think - so same day historical targets would be at 5 days, 12 days, etc? 

So, there are a few things that I think should be of interest to our model:

- The entire datablock's worth of targets for that prediction_id. Why not?
- The entire datablock's worth of targets for the complement of production/consumption. I've noticed that consumption is somewhat affected by production, so why not include the data for the model? I don't think production is affected by consumption though.
- Past values for the hour for 1-14 days previous                   + (both production and consumption?)
- Average for the same hour, 7 day, 14 day                          + means/stds? + (both production and consumption?)
- Daily Past averages for the last 7 days                           + means/stds? + (both production and consumption?)
- Average for the same hour, for the entire county                  + means/stds? + (both production and consumption?)
- Daily Past averages for the last 7 days, for the entire county    + means/stds? + (both production and consumption?)

+ Averages across entire product_type ?

In [101]:
class RevealedTargetsLoader:
    """Stores and processes the data from the revealed_targets df"""
    def __init__(self, df):
        self.df = df
        self.recent_excerpt = self.init_recent_excerpt(df)

    def init_recent_excerpt(self, df):
        """Gets only the most recent data from the df, to reduce processing time for future feature creation"""
        pass

    def add_test_df(self, df):
        """adds the test df to the data"""
        pass

    def _create_entire_data_block_features(self, df:pd.DataFrame):
        # ensure df is sorted by date
        # group by create a column for each hour of the day, odd numbers are opposite of is_consumption for the row
        df['unique_id'] = df.groupby(['data_block_id', 'prediction_unit_id']).cumcount()
        #create pivot table to create a column for each historical target
        pivot_df = df.pivot_table(index=['data_block_id', 'prediction_unit_id'], 
                          columns='unique_id', 
                          values='target', 
                          fill_value=0)
        # reset index to flatten the df and create separate columns
        pivot_df = pivot_df.reset_index()
        #name columns
        pivot_df.columns = [(f'target_{int(col/2)}' if col%2 == 0 else f'target_{int((col-1)/2)}_complement') if isinstance(col, int) else col for col in pivot_df.columns]
        return pivot_df
    
    def _create_past_hourly_values(self, df:pd.DataFrame):
        # ensure df is sorted by date

        df = df.set_index('datetime')
        df['id'] = df['prediction_unit_id'].astype(str) + '_' + df['is_consumption'].astype(str)


        # Every day from 1-12 days ago
        periods = [24, 48, 72, 96, 120, 144, 168, 192, 216, 240, 264, 288]

        # Initialize a list to store the shifted DataFrames
        shifted_dfs = []

        # Loop through each period and create a shifted DataFrame
        for period in periods:
            shifted_df = df.groupby('id')['target'].shift(period)
            shifted_df.name = f'target_lag_{period}'  # Rename the column
            # display(shifted_df)
            shifted_dfs.append(shifted_df)

        # Concatenate all shifted DataFrames with the original DataFrame
        # Note: I should cut these columns down further to only the ones I need for merging.
        # result_df = pd.concat([df[['county', 'is_business', 'product_type', 'is_consumption', 'data_block_id', 'row_id', 'prediction_unit_id']]] + shifted_dfs, axis=1)
        result_df = pd.concat([df] + shifted_dfs, axis=1)
        result_df = result_df.reset_index()
        return result_df
    
    def _create_sma_values(self, df:pd.DataFrame):
        # ensure df is sorted by date
        df = df.set_index('datetime')
        df['id'] = df['prediction_unit_id'].astype(str) + '_' + df['is_consumption'].astype(str)

        windows = [8, 12, 24, 48, 24*7, 24*14]
        windowed_dfs = []

        other_df = df.reset_index()
        # Loop through each period and create avgs
        for window in windows:
            my_df = df.groupby('id')['target'].rolling(window=window, min_periods=1).mean()
            my_df = my_df.reset_index()
            my_df = my_df.rename(columns={'target': f'target_{window}h_sma'})
            other_df = pd.merge(other_df, my_df, on=['id', 'datetime'])

        # result_df = pd.concat(windowed_dfs, axis=1)
        return other_df.drop('id', axis=1)
    
    def _create_hourly_sma_values(self, df:pd.DataFrame):
        # ensure df is sorted by date
        
        df['id'] = df['prediction_unit_id'].astype(str) + '_' + df['is_consumption'].astype(str) + '_' + df['datetime'].dt.hour.astype(str)
        df = df.set_index('datetime')
        windows = [24*7, 24*14]
        windowed_dfs = []

        other_df = df.reset_index()
        # Loop through each period and create avgs
        for window in windows:
            my_df = df.groupby('id')['target'].rolling(window=window, min_periods=1).mean()
            my_df = my_df.reset_index()
            my_df = my_df.rename(columns={'target': f'target_{window}h_hsma'})
            other_df = pd.merge(other_df, my_df, on=['id', 'datetime'])

        # result_df = pd.concat(windowed_dfs, axis=1)
        return other_df.drop('id', axis=1)
    
    def _create_county_sma_values(self, df:pd.DataFrame):
        # ensure df is sorted by date
        df = df.set_index('datetime')
        df['id'] = df['county'].astype(str) + '_' + df['is_consumption'].astype(str)
        
        windows = [24, 48, 24*7]
        
        # Loop through each period and create avgs
        df = df
        for window in windows:
            df[f'target_{window}h_csma'] = df.groupby('id')['target'].transform(lambda x: x.rolling(window=window, min_periods=1).mean())

        # result_df = pd.concat(windowed_dfs, axis=1)
        return df.drop(['id', 'target'], axis=1)
    
    def _create_hourly_county_sma_values(self, df:pd.DataFrame):
        # ensure df is sorted by date
        df['id'] = df['county'].astype(str) + '_' + df['is_consumption'].astype(str) + '_' + df['datetime'].dt.hour.astype(str)
        df = df.set_index('datetime')
        windows = [24*7, 24*14]
        # Loop through each period and create avgs
        df = df
        for window in windows:
            df[f'target_{window}h_hcsma'] = df.groupby('id')['target'].transform(lambda x: x.rolling(window=window, min_periods=1).mean())

        return df.drop(['id', 'target'], axis=1)
    
    def _merge(self, df1:pd.DataFrame, df2:pd.DataFrame, merge_cols=['datetime', 'county', 'is_business', 'product_type', 'target', 'is_consumption', 'data_block_id', 'row_id', 'prediction_unit_id']):
        # Assuming df and hourly_df are your DataFrames

        # Create unique_id in both DataFrames
        df1['unique_id'] = df1[merge_cols].astype(str).agg('_'.join, axis=1)
        df2['unique_id'] = df2[merge_cols].astype(str).agg('_'.join, axis=1)
        df2 = df2.drop(merge_cols, axis=1)

        # Merge on unique_id
        merged_df = pd.merge(df1, df2, on='unique_id', how='inner')

        # Drop the unique_id column
        merged_df.drop('unique_id', axis=1, inplace=True)

        # Drop the duplicated columns from one of the DataFrames
        # Assuming the columns in hourly_df are the ones to be dropped
        # for col in merge_cols:
        #     if col + '_y' in merged_df:
        #         merged_df.drop(col + '_y', axis=1, inplace=True)
        #         merged_df.rename(columns={col + '_x': col}, inplace=True)
        return merged_df

    def create_features(self, df:pd.DataFrame):
        """Creates the features"""
        # First, ensure df is sorted
        df = df.sort_values(['datetime', 'prediction_unit_id', 'is_consumption'])
        # display(df)

        #### Entire datablocks worth of targets for that prediction_id - both production and consumption
        datablock_df = self._create_entire_data_block_features(df)
        # display(datablock_df)
        

        #### Past values for the same hour
        hourly_df = self._create_past_hourly_values(df)
        
        # df =  pd.merge(df, hourly_df, on=['datetime','county','is_business','product_type','target','is_consumption','data_block_id','row_id', 'prediction_unit_id'])

        #### simple moving average
        rolling_avg_df = self._create_sma_values(df)
        #join
        
        # df =  pd.merge(df, rolling_avg_df, on=['datetime','county','is_business','product_type','target','is_consumption','data_block_id','row_id', 'prediction_unit_id'])

        #### simple moving average for only same hourly values
        hourly_rolling_avg_df = self._create_hourly_sma_values(df)
        #join
        
        # df =  pd.merge(df, hourly_rolling_avg_df, on=['datetime','county','is_business','product_type','target','is_consumption','data_block_id','row_id', 'prediction_unit_id'])

        #### simple moving average for county
        county_rolling_avg_df = self._create_county_sma_values(df)
        #join
        
        # df =  pd.merge(df, county_rolling_avg_df, on=['datetime','county','is_business','product_type','is_consumption','data_block_id','row_id', 'prediction_unit_id'])

        #### simple moving average for county for only same hourly values
        county_hourly_rolling_avg_df = self._create_hourly_county_sma_values(df)
        #join
        
        # df =  pd.merge(df, county_hourly_rolling_avg_df, on=['datetime','county','is_business','product_type','is_consumption','data_block_id','row_id', 'prediction_unit_id'])

        #join
        df = pd.merge(df, datablock_df, on=['data_block_id', 'prediction_unit_id'])
        #join
        df = self._merge(df, hourly_df)
        df = self._merge(df, rolling_avg_df)
        df = self._merge(df, hourly_rolling_avg_df)
        df = self._merge(df, county_rolling_avg_df, merge_cols=['county', 'is_business', 'product_type', 'is_consumption', 'data_block_id', 'row_id', 'prediction_unit_id'])
        df = self._merge(df, county_hourly_rolling_avg_df, merge_cols=['county', 'is_business', 'product_type', 'is_consumption', 'data_block_id', 'row_id', 'prediction_unit_id'])

        #drop some cols that made it though
        # df = df.drop(['unique_id', 'id'], axis=1)
        # print(df.columns)
        return df


datablock = 274
revealed_targets = pd.read_csv('data/train.csv', parse_dates=['datetime'])
revealed_targets = revealed_targets[revealed_targets.data_block_id < datablock-2].copy()
revealed_targets['data_block_id'] += 2
# display(revealed_targets)

ob = RevealedTargetsLoader(revealed_targets)
# tdf = ob._create_entire_data_block_features(revealed_targets)
# tdf = ob._create_past_hourly_values(revealed_targets)
# tdf = ob._create_sma_values(revealed_targets)
# tdf = ob._create_hourly_sma_values(revealed_targets)
# tdf = ob._create_county_sma_values(revealed_targets)
# tdf = ob._create_hourly_county_sma_values(revealed_targets)
# tdf
feats = ob.create_features(revealed_targets)
feats.sort_values('row_id')


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,id_x,...,target_48h_sma,target_168h_sma,target_336h_sma,target_168h_hsma,target_336h_hsma,target_24h_csma,target_48h_csma,target_168h_csma,target_168h_hcsma,target_336h_hcsma
0,0,0,1,0.713,0,2021-09-01 00:00:00,2,0,0,0_0_0,...,0.713000,0.713000,0.713000,0.713000,0.713000,0.713000,0.713000,0.713000,0.713000,0.713000
1,0,0,1,96.590,1,2021-09-01 00:00:00,2,1,0,0_1_0,...,96.590000,96.590000,96.590000,96.590000,96.590000,96.590000,96.590000,96.590000,96.590000,96.590000
48,0,0,2,0.000,0,2021-09-01 00:00:00,2,2,1,0_0_0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.356500,0.356500,0.356500,0.356500,0.356500
49,0,0,2,17.314,1,2021-09-01 00:00:00,2,3,1,0_1_0,...,17.314000,17.314000,17.314000,17.314000,17.314000,56.952000,56.952000,56.952000,56.952000,56.952000
96,0,0,3,2.904,0,2021-09-01 00:00:00,2,4,2,0_0_0,...,2.904000,2.904000,2.904000,2.904000,2.904000,1.205667,1.205667,1.205667,1.205667,1.205667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844655,15,1,0,108.908,1,2022-05-30 23:00:00,273,844795,64,15_1_23,...,133.171604,141.056583,143.589423,162.695589,161.846757,136.184792,143.070437,131.021595,119.710565,127.176765
844414,15,1,1,0.000,0,2022-05-30 23:00:00,273,844796,59,15_0_23,...,76.201917,76.541125,75.369533,0.000000,0.012868,4.120000,55.986938,77.921196,0.653500,0.739021
844415,15,1,1,40.896,1,2022-05-30 23:00:00,273,844797,59,15_1_23,...,31.517104,35.347310,38.828467,57.173470,51.560629,125.378042,134.915812,128.677655,118.110208,126.984560
844462,15,1,3,1.000,0,2022-05-30 23:00:00,273,844798,60,15_0_23,...,163.585396,147.296375,148.308545,0.273042,0.178191,3.690250,54.590958,77.373667,0.659452,0.741997


## Client Features

I am doing no transformations with client features

In [5]:
class ClientLoader:
    """Stores the data in the client file"""
    def __init__(self, df):
        self.df = df
        self.recent_excerpt = self.init_recent_excerpt(df)

    def init_recent_excerpt(self, df):
        """Gets only the most recent data from the df, to reduce processing time for future feature creation"""
        pass

    def add_test_df(self, df):
        """adds the test df to the data"""
        pass  

## Historical Weather

We are provided with historical weather data for weather stations within counties. A couple of ideas for this weather data:

- Latest historical weather data for the county (avg if more than 1 station)
- Avg weather data for a county 24h, 48h + var
- Avg weather data for all of the country + var


In [8]:
class HistoricalWeatherLoader:
    """Stores the data in the client file"""
    def __init__(self, df):
        self.df = df
        self.recent_excerpt = self.init_recent_excerpt(df)
        self.weather_mapping = self.init_weather_mapping()
        self.weather_features = ['temperature','dewpoint','rain','snowfall','surface_pressure','cloudcover_total','cloudcover_low','cloudcover_mid','cloudcover_high','windspeed_10m','winddirection_10m','shortwave_radiation','direct_solar_radiation','diffuse_radiation']

    def init_weather_mapping(self):
        # https://www.kaggle.com/code/tsunotsuno/enefit-eda-baseline/notebook#Baseline
        county_point_map = {
            0: (59.4, 24.7), # "HARJUMAA"
            1 : (58.8, 22.7), # "HIIUMAA"
            2 : (59.1, 27.2), # "IDA-VIRUMAA"
            3 : (58.8, 25.7), # "JÄRVAMAA"
            4 : (58.8, 26.2), # "JÕGEVAMAA"
            5 : (59.1, 23.7), # "LÄÄNE-VIRUMAA"
            6 : (59.1, 23.7), # "LÄÄNEMAA"
            7 : (58.5, 24.7), # "PÄRNUMAA"
            8 : (58.2, 27.2), # "PÕLVAMAA"
            9 : (58.8, 24.7), # "RAPLAMAA"
            10 : (58.5, 22.7),# "SAAREMAA"
            11 : (58.5, 26.7),# "TARTUMAA"
            12 : (58.5, 25.2),# "UNKNOWNN" (center of the map)
            13 : (57.9, 26.2),# "VALGAMAA"
            14 : (58.2, 25.7),# "VILJANDIMAA"
            15 : (57.9, 27.2) # "VÕRUMAA"
        }
        # Convert the dictionary to a list of tuples
        data = [(county_code, lat, lon) for county_code, (lat, lon) in county_point_map.items()]

        # Create DataFrame
        df = pd.DataFrame(data, columns=['county', 'latitude', 'longitude'])
        
        return df

    def init_recent_excerpt(self, df):
        """Gets only the most recent data from the df, to reduce processing time for future feature creation"""
        pass

    def add_test_df(self, df):
        """adds the test df to the data"""
        pass

    def _add_latest_weather(self, df:pd.DataFrame):
        pass

    def create_features(self, df:pd.DataFrame):
        # map l+l to county
        df = df.merge(self.weather_mapping, how='inner', on=('latitude', 'longitude'))
        # print(weather_features)

        # Apply the function
        county_mean = self.add_24h_mean_var(df, self.weather_features)    
        country_mean = self.add_24h_mean_var_estonia(df, self.weather_features)
           
        latest = self._add_latest_weather(df)
        pass

In [31]:
datablock = 274
df = pd.read_csv('data/historical_weather.csv', parse_dates=['datetime'])
df = df[df.data_block_id < datablock].copy()
# display(revealed_targets)

# ob = RevealedTargetsLoader(revealed_targets)
# tdf = ob._create_entire_data_block_features(revealed_targets)
# tdf = ob._create_past_hourly_values(revealed_targets)
# tdf = ob._create_sma_values(revealed_targets)
# tdf = ob._create_hourly_sma_values(revealed_targets)
# tdf = ob._create_county_sma_values(revealed_targets)
# tdf = ob._create_hourly_county_sma_values(revealed_targets)
# tdf

ob = HistoricalWeatherLoader(df)

df = df.merge(ob.weather_mapping, on=('latitude', 'longitude'))

# df.groupby(['county', 'datetime'])

max_datetime_rows = df.loc[df.groupby(['data_block_id', 'county'])['datetime'].idxmax()]
max_datetime_rows
# df
# df

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id,county
98095,2021-09-01 10:00:00,13.6,8.7,0.0,0.0,1008.9,12,7,9,0,6.305556,331,396.0,311.0,85.0,59.4,24.7,1.0,0
52322,2021-09-01 10:00:00,14.3,8.4,0.0,0.0,1014.4,53,28,0,94,7.777778,340,377.0,223.0,154.0,58.8,22.7,1.0,1
91556,2021-09-01 10:00:00,14.5,10.9,0.0,0.0,1004.2,50,46,15,0,4.916667,322,318.0,196.0,122.0,59.1,27.2,1.0,2
65400,2021-09-01 10:00:00,14.4,10.5,0.0,0.0,1003.3,38,26,24,0,4.861111,323,362.0,224.0,138.0,58.8,25.7,1.0,3
71939,2021-09-01 10:00:00,14.5,10.6,0.0,0.0,1001.1,62,49,30,0,5.194444,320,375.0,235.0,140.0,58.8,26.2,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52311,2022-05-31 10:00:00,12.1,10.0,0.0,0.0,1001.6,96,72,51,1,5.472222,75,221.0,81.0,140.0,58.5,26.7,273.0,11
45772,2022-05-31 10:00:00,11.4,10.2,0.2,0.0,1002.3,100,98,59,3,5.333333,64,173.0,21.0,152.0,58.5,25.2,273.0,12
6538,2022-05-31 10:00:00,11.3,9.9,0.1,0.0,1001.5,100,89,71,1,6.444444,66,130.0,14.0,116.0,57.9,26.2,273.0,13
19616,2022-05-31 10:00:00,10.8,10.0,0.1,0.0,994.5,100,100,65,0,6.000000,64,118.0,3.0,115.0,58.2,25.7,273.0,14


In [57]:
datablock = 274
df = pd.read_csv('data/train.csv', parse_dates=['datetime'])
df = df[df.data_block_id < datablock-2].copy()
df['data_block_id'] += 2

df = df.sort_values(['datetime', 'prediction_unit_id', 'is_consumption'])
df = df.set_index('datetime')
df['id'] = df['prediction_unit_id'].astype(str) + '_' + df['is_consumption'].astype(str)

windows = [8, 12, 24, 48, 24*7, 24*14]
other_df = df.reset_index()
# Loop through each period and create avgs
for window in windows:
    df[f'target_{window}h_sma'] = df.groupby('id')['target'].transform(lambda x: x.rolling(window=window, min_periods=1).mean())
    # my_df = my_df.reset_index()
    # my_df = my_df.rename(columns={'target': f'target_{window}h_sma'})
    # display(my_df)
    # break
    # other_df = pd.merge(other_df, my_df, on=['id', 'datetime'])
    # display(my_df)
    # df[f'{window}h_sma'] = my_df

df

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,id,target_8h_sma,target_12h_sma,target_24h_sma,target_48h_sma,target_168h_sma,target_336h_sma
datetime,,,,,,,,,,,,,,,
2021-09-01 00:00:00,0,0,1,0.713,0,2,0,0,0_0,0.713000,0.713000,0.713000,0.713000,0.713000,0.713000
2021-09-01 00:00:00,0,0,1,96.590,1,2,1,0,0_1,96.590000,96.590000,96.590000,96.590000,96.590000,96.590000
2021-09-01 00:00:00,0,0,2,0.000,0,2,2,1,1_0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2021-09-01 00:00:00,0,0,2,17.314,1,2,3,1,1_1,17.314000,17.314000,17.314000,17.314000,17.314000,17.314000
2021-09-01 00:00:00,0,0,3,2.904,0,2,4,2,2_0,2.904000,2.904000,2.904000,2.904000,2.904000,2.904000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-30 23:00:00,2,1,1,41.338,1,273,844691,65,65_1,43.745750,42.385583,45.605417,47.576250,47.422845,47.986027
2022-05-30 23:00:00,4,1,1,0.000,0,273,844708,66,66_0,22.252375,51.620667,45.153542,42.830354,40.564274,47.786018
2022-05-30 23:00:00,4,1,1,88.192,1,273,844709,66,66_1,85.601125,68.667833,72.642625,67.988542,75.066607,78.876830


In [48]:
datablock = 274
df = pd.read_csv('data/train.csv', parse_dates=['datetime'])
df = df[df.data_block_id < datablock-2].copy()
df['data_block_id'] += 2

df = df.sort_values(['datetime', 'prediction_unit_id', 'is_consumption'])
df = df.set_index('datetime')
df['id'] = df['prediction_unit_id'].astype(str) + '_' + df['is_consumption'].astype(str)



periods = [24, 48, 72, 96, 120, 144, 168, 192, 216, 240, 268, 292]

# Initialize a list to store the shifted DataFrames
shifted_dfs = []

# Loop through each period and create a shifted DataFrame
for period in periods:
    shifted_df = df.groupby('id')['target'].shift(period)
    shifted_df.name = f'target_lag_{period}'  # Rename the column
    shifted_dfs.append(shifted_df)

# Concatenate all shifted DataFrames with the original DataFrame
result_df = pd.concat([df] + shifted_dfs, axis=1)
result_df

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,id,target_lag_24,...,target_lag_72,target_lag_96,target_lag_120,target_lag_144,target_lag_168,target_lag_192,target_lag_216,target_lag_240,target_lag_268,target_lag_292
datetime,,,,,,,,,,,,,,,,,,,,,
2021-09-01 00:00:00,0,0,1,0.713,0,2,0,0,0_0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-01 00:00:00,0,0,1,96.590,1,2,1,0,0_1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-01 00:00:00,0,0,2,0.000,0,2,2,1,1_0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-01 00:00:00,0,0,2,17.314,1,2,3,1,1_1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-01 00:00:00,0,0,3,2.904,0,2,4,2,2_0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-30 23:00:00,2,1,1,41.338,1,273,844691,65,65_1,49.310,...,57.566,45.138,37.696,40.427,46.279,50.540,51.677,51.263,44.959,33.549
2022-05-30 23:00:00,4,1,1,0.000,0,273,844708,66,66_0,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,8.014,10.065
2022-05-30 23:00:00,4,1,1,88.192,1,273,844709,66,66_1,89.906,...,79.231,92.174,95.418,91.396,73.996,92.429,93.970,103.111,92.670,105.669


In [66]:
import pandas as pd
import datetime as dt
import numpy as np

import pandas as pd
import datetime as dt
import numpy as np
import time

class TrainDataProcessor:
    """I am rewriting this training data processor to process a few more variables differently."""

    def __init__(self, train, revealed_targets, client, historical_weather,
                 forecast_weather, electricity_prices, gas_prices, for_testing=False,
                add_log_cols=False):
        self.add_log_cols = add_log_cols
        self.test_orig_dfs = self.get_test_orig_dfs([train, revealed_targets, client, historical_weather,
                 forecast_weather, electricity_prices, gas_prices])
        
        self.weather_mapping = self.init_weather_mapping()
        
        if not for_testing:
            self.train = self.init_train(train)
            display(self.train)
            self.revealed_targets = self.init_revealed_targets(revealed_targets)
            self.client = self.init_client(client)
            
            self.historical_weather = self.init_historical_weather(historical_weather)
            self.forecast_weather = self.init_forecast_weather(forecast_weather)
            self.electricity_prices = self.init_electricity(electricity_prices)
            self.gas_prices = self.init_gas_prices(gas_prices)
            
            self.df_all_cols = self.join_data(self.train, self.revealed_targets, self.client, self.historical_weather, self.forecast_weather, self.electricity_prices, self.gas_prices)
            if self.add_log_cols:
                self.df_all_cols = self.create_log_cols(self.df_all_cols)
            self.df = self.remove_cols(self.df_all_cols)
            # self.target = self.df['target']
            # self.df = self.df.drop('target', axis=1)
            
        
    def get_test_orig_dfs(self, dfs):
        for i, df in enumerate(dfs):
            if 'datetime' in df.columns:
                df['datetime'] = pd.to_datetime(df.datetime)
                col = 'datetime'
            if 'prediction_datetime' in df.columns:
                df['prediction_datetime'] = pd.to_datetime(df.prediction_datetime)
                col = 'prediction_datetime'
            if 'forecast_date' in df.columns:
                df['forecast_date'] = pd.to_datetime(df['forecast_date'])
                col = 'forecast_date'
            if 'forecast_datetime' in df.columns:
                df['forecast_datetime'] = pd.to_datetime(df['forecast_datetime'])
                col = 'forecast_datetime'
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df.date).dt.date
                col = 'date'

            test_date = df[col].iloc[-1]  # Assuming test is a DataFrame
            start_date = test_date - pd.Timedelta(days=14)
            historical_subset = df[df[col] >= start_date]
            dfs[i] = historical_subset.copy()
        return dfs
    
    def trim_test_orig_dfs(self, dfs):
        for i, df in enumerate(dfs):
            if 'datetime' in df.columns:
                col = 'datetime'
            if 'prediction_datetime' in df.columns:
                col = 'prediction_datetime'
            if 'forecast_date' in df.columns:
                col = 'forecast_date'
            if 'forecast_datetime' in df.columns:
                col = 'forecast_datetime'
            if 'date' in df.columns:
                col = 'date'

            test_date = df[col].iloc[-1]  # Assuming test is a DataFrame
            #set cutoff date to a little longer than 14 for a buffer. Shoudn't need the buffer tho honestly
            start_date = test_date - pd.Timedelta(days=20)
            historical_subset = df[df[col] >= start_date]
            dfs[i] = historical_subset.copy()
        return dfs
        
    def init_train(self, df):
        """Prepares the training data for model training."""
        try:
            df['datetime'] = pd.to_datetime(df.datetime)
        except Exception as e:
            df['datetime'] = pd.to_datetime(df.prediction_datetime)
        df['date'] = df.datetime.dt.date
        
        #for test dfs, set index to row id
        df.index = df.row_id.tolist()
            
        # df = self.get_data_block_id(df, 'datetime')
        return df
    
    def add_electricity_lag_features(self, df):
        ##### mean from entire last week
        df.set_index('datetime', inplace=True)
        # Use rolling to calculate mean price of the last week
        # The window is 7 days, min_periods can be set as per requirement
        # 'closed' determines which side of the interval is closed; it can be 'right' or 'left'
        # display(df)
        df['mean_euros_per_mwh_last_week'] = df['euros_per_mwh'].rolling(window='7D', min_periods=1, closed='right').mean()
        # Shift the results to align with the requirement of lagging
        df['mean_euros_per_mwh_last_week'] = df['mean_euros_per_mwh_last_week'].shift()
        
        ##### mean from last week this hour only
        # Extract hour from datetime
        df['hour'] = df.index.hour

        # Group by hour and apply rolling mean for each group
        hourly_groups = df.groupby('hour')
        dff = hourly_groups['euros_per_mwh'].rolling(window='7D', min_periods=1, closed='right').mean()#.shift()#.reset_index(level=0, drop=True)
        dff = dff.reset_index().set_index('datetime').groupby('hour')['euros_per_mwh'].shift()
        dff = dff.rename('mean_euros_per_mwh_same_hour_last_week')
        df = df.join(dff)
        #### yesterday's power price
        df['yesterdays_euros_per_mwh'] = df['euros_per_mwh'].shift(24)
        
        ### 24h average
        # Calculate the 24-hour rolling average
        df['euros_per_mwh_24h_average_price'] = df['euros_per_mwh'].rolling(window=24, min_periods=1).mean()

        # Resetting the index if needed
        df.reset_index(inplace=True)
        df = df.drop(['forecast_date', 'origin_date', 'hour'], axis=1)
        return df

    def init_electricity(self, df):
        ## LAG = 1 Day
        ## Move forecast datetime ahead by 1 day
        ## change name to datetime
        df['datetime'] = pd.to_datetime(df['forecast_date'])
        df['datetime'] = df['datetime'] + dt.timedelta(days=1)
        # df = self.get_data_block_id(df, 'datetime')
        df = self.add_electricity_lag_features(df)
        return df
    
    def add_historical_weather_lag_features(self, df):
        ##### LATEST WEATHER
        def add_latest_weather(df):
            # Assuming df is your original DataFrame
            # Step 1: Convert datetime to a Datetime Object
            df['datetime'] = pd.to_datetime(df['datetime'])
            df.set_index('datetime', inplace=True)

            # Step 2: Sorting the Data
            df.sort_values(by=['datetime', 'latitude', 'longitude'], inplace=True)

            # Step 3: Creating a Unique Identifier for each location
            df['location_id'] = df['latitude'].astype(str) + '_' + df['longitude'].astype(str)

            # Step 4: Filtering for 10:00 AM Entries
            df.reset_index(inplace=True)
            df_10am = df[df['datetime'].dt.hour == 10]
            df_10am.set_index('datetime', inplace=True)

            # Step 5: Shifting the Features by 1 day
            lagged_features = df_10am.groupby('location_id').shift(periods=1, freq='D')
            
            # grouped = lagged_features.groupby('county')
            # lagged_features = grouped[weather_features].mean()
            
            
            # Renaming columns to indicate lag
            lagged_features = lagged_features.add_suffix('_hw_lagged')
            lagged_features['location_id'] = lagged_features['location_id_hw_lagged']
            lagged_features.reset_index(inplace=True)
            lagged_features['date'] = lagged_features.datetime.dt.date

            df['date'] = df.datetime.dt.date
            return lagged_features
            # Step 6: Merging Lagged Features with Original DataFrame
            df = df.merge(lagged_features, on=['date', 'location_id'], how='left', suffixes=('', '_hw_lagged'))
            return df
        
        ##### mean from last day
        def add_24h_mean_var(df, weather_features):
            # Calculate the start and end times for each row
            # df['start_time'] = pd.to_datetime(df['datetime'].dt.date) - pd.Timedelta(days=2) + pd.Timedelta(hours=11)
            # df['end_time'] = pd.to_datetime(df['datetime'].dt.date) - pd.Timedelta(days=1) + pd.Timedelta(hours=10)
            # df['time_code'] = df['start_time'].astype(str) +'_' + df['end_time'].astype(str) + '_' + df['latitude'].astype(str) + '_' + df['longitude'].astype(str)
            # print(df.time_code)

            # Create a helper column for grouping
            # If the time is before 11:00 AM, subtract a day
            df['group'] = df['datetime'].apply(lambda dt: dt if dt.time() >= pd.to_datetime('11:00').time() else dt - pd.Timedelta(days=1))
            df['group'] = df['group'].dt.date  # Keep only the date part for grouping
            df['group'] = (pd.to_datetime(df['group']) + pd.Timedelta(hours=11)).astype(str) + '_' + (pd.to_datetime(df['group']) + pd.Timedelta(days=1, hours=10)).astype(str) + '_' + df['latitude'].astype(str) + '_' + df['longitude'].astype(str)

            # Now group by this new column
            grouped = df.groupby('group')
            means = grouped[weather_features].mean()
            variances = grouped[weather_features].var()

            # Merge means and variances into the original DataFrame
            my_df = df.merge(means, on='group', suffixes=('', '_hw_means'), how='left')
            my_df = my_df.merge(variances, on='group', how='left', suffixes=('', '_hw_variances'))

            return my_df
        
        ##### mean from last day all estonia
        def add_24h_mean_var_estonia(df, weather_features):
            # Calculate the start and end times for each row
            # df['start_time'] = pd.to_datetime(df['datetime'].dt.date) - pd.Timedelta(days=2) + pd.Timedelta(hours=11)
            # df['end_time'] = pd.to_datetime(df['datetime'].dt.date) - pd.Timedelta(days=1) + pd.Timedelta(hours=10)
            # df['time_code'] = df['start_time'].astype(str) +'_' + df['end_time'].astype(str)
            # print(df.time_code)

            # Create a helper column for grouping
            # If the time is before 11:00 AM, subtract a day
            df['group'] = df['datetime'].apply(lambda dt: dt if dt.time() >= pd.to_datetime('11:00').time() else dt - pd.Timedelta(days=1))
            df['group'] = df['group'].dt.date  # Keep only the date part for grouping
            df['group'] = (pd.to_datetime(df['group']) + pd.Timedelta(hours=11)).astype(str) + '_' + (pd.to_datetime(df['group']) + pd.Timedelta(days=1, hours=10)).astype(str) + '_' + df['latitude'].astype(str) + '_' + df['longitude'].astype(str)

            # Now group by this new column
            grouped = df.groupby('group')
            means = grouped[weather_features].mean()
            variances = grouped[weather_features].var()

            # Merge means and variances into the original DataFrame
            my_df = df.merge(means, on='group', suffixes=('', '_hw_means_estonia'), how='left')
            my_df = my_df.merge(variances, on='group', how='left', suffixes=('', '_hw_variances_estonia'))

            return my_df

        df['datetime'] = pd.to_datetime(df['datetime'])
        # 
        weather_features = ['temperature','dewpoint','rain','snowfall','surface_pressure','cloudcover_total','cloudcover_low','cloudcover_mid','cloudcover_high','windspeed_10m','winddirection_10m','shortwave_radiation','direct_solar_radiation','diffuse_radiation']
        # print(weather_features)

        # Apply the function
        df = add_24h_mean_var(df, weather_features)    
        df = add_24h_mean_var_estonia(df, weather_features)
           
        latest = add_latest_weather(df)
        df = df.merge(latest, on=['date', 'location_id'], how='left', suffixes=('', '_hw_lagged'))
        
        return df

    def init_historical_weather(self, df):
        ## LAG: From 11:00 AM 2 days ago to 10:00 AM 1 day ago
        ## What to do? Give most recent weather forecast? Give average over the last day?
        """
        Processes the historical weather data.
        """
        df['datetime'] = pd.to_datetime(df.datetime)
        df = df.drop('data_block_id', axis=1)
        
        
        df = self.add_historical_weather_lag_features(df)
        
        df = df.merge(self.weather_mapping, how='inner', on=('latitude', 'longitude'))
        
        return df

    def init_forecast_weather(self, df):
        ## LAG: DON't ADJUST
        ##      The forecast is from yesterday, but can forecast today, which is 22 hours ahead
        ## Drop any columns where:
        ##                        hours_ahead < 22 and hours_ahead > 45
        ## Then rename forecast_datetime to datetime and join on datetime
        """
        Processes the forecast weather data.
        """
        df['datetime'] = pd.to_datetime(df['forecast_datetime'])
        # keep only datetimes from our relevant period
        df = df[(df['hours_ahead'] < 46) & (df['hours_ahead'] > 21)]
        df['datetime'] = df['datetime'] + dt.timedelta(days=1)
        df = df.merge(self.weather_mapping, how='inner', on=('latitude', 'longitude'))
        return df
    
    def add_gas_prices_lag_features(self, df):
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)

        # Sort the DataFrame by date, if it's not already sorted
        df.sort_index(inplace=True)

        # Calculate rolling averages for different time windows
        df['lowest_price_3d_avg'] = df['lowest_price_per_mwh'].rolling(window=3).mean()
        df['highest_price_3d_avg'] = df['highest_price_per_mwh'].rolling(window=3).mean()

        df['lowest_price_7d_avg'] = df['lowest_price_per_mwh'].rolling(window=7).mean()
        df['highest_price_7d_avg'] = df['highest_price_per_mwh'].rolling(window=7).mean()

        df['lowest_price_14d_avg'] = df['lowest_price_per_mwh'].rolling(window=14).mean()
        df['highest_price_14d_avg'] = df['highest_price_per_mwh'].rolling(window=14).mean()

        # Reset the index if you want the 'date' column back
        df.reset_index(inplace=True)
        return df

    def init_gas_prices(self, df):
        ## LAG: 1 DAY
        ## Predictions are made from 2 days ago and predict for yesterday
        ## add one day to forecast_date
        ## Rename forecast_date to date, join on date
        """
        Processes the gas prices data.
        Implement the logic to handle gas prices data processing here.
        """
        df['date'] = pd.to_datetime(df['forecast_date']).dt.date
        df['date'] = df['date'] + dt.timedelta(days=1)
        df = self.add_gas_prices_lag_features(df)
        return df
    
    def add_revealed_target_features(self, df):
        df['datetime'] = pd.to_datetime(df['datetime'])
        #for test dfs, set index to row id
        df.index = df.row_id.tolist()
        df['hour'] = df.datetime.dt.hour
        df['day'] = df.datetime.dt.dayofweek
        df.set_index('datetime', inplace=True)
        
        # let me add some new features here too
        # Adding lag features
        # Step 2: Sorting the Data
        df.sort_values(by=['datetime'], inplace=True)

        # Step 3: Creating a Unique Identifier for each location
        df['id'] = df['county'].astype(str) + '_' + df['is_business'].astype(str) + '_' + df['product_type'].astype(str) + '_' + df['is_consumption'].astype(str)
        lagged_features = []
        lagged_hours = []
        ### Defining lagged target features
        display('df')
        display(df)

        for lag_hours in range(1, 24):
            lagged_feature = df.groupby('id').shift(periods=lag_hours, freq='H')
            lagged_features.append(lagged_feature)
            lagged_hours.append(lag_hours)
            if lag_hours == 1:
                display('lagged_feature 1')
                display(lagged_feature)

        for lag_hours in ([i*24 for i in range(1,8)] + [24*11, 24*12]):
            lagged_feature = df.groupby('id').shift(periods=lag_hours, freq='H')
            lagged_features.append(lagged_feature)
            lagged_hours.append(lag_hours)
            if lag_hours == 1:
                display('lagged_feature 2')
                display(lagged_feature)
            
        df.reset_index(inplace=True)
        for lagged_feature, lag_hours in zip(lagged_features, lagged_hours):
            lagged_feature.reset_index(inplace=True)
            lagged_feature.dropna(inplace=True)
            df = df.merge(lagged_feature[['datetime', 'target', 'id']], on=['id', 'datetime'], how='left', suffixes=('', f'_lag_{lag_hours}h'))

        df.set_index('datetime', inplace=True)
        display('df 2')
        display(df)
        

        window_size = 7
        # Group by the specified columns and then apply the rolling mean
        grouped = df.groupby(['county', 'is_business', 'product_type', 'is_consumption'])
        df['target_rolling_avg_24h'] = grouped['target'].transform(lambda x: x.rolling(window=24, min_periods=1).mean())

        grouped = df.groupby(['county', 'is_business', 'product_type', 'is_consumption', 'hour'])
        df['target_rolling_avg_hour_7d'] = grouped['target'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

        # grouped = df.groupby(['county', 'is_business', 'product_type', 'is_consumption', 'hour', 'day'])
        # df['target_rolling_avg_hour_hour_day_4w'] = grouped['target'].transform(lambda x: x.rolling(window=4, min_periods=1).mean())

        grouped = df.groupby(['county', 'is_business', 'is_consumption'])
        df['target_rolling_allp_avg_24h'] = grouped['target'].transform(lambda x: x.rolling(window=24, min_periods=1).mean())

        grouped = df.groupby(['county', 'is_business', 'is_consumption', 'hour'])
        df['target_rolling_allp_avg_hour_7d'] = grouped['target'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

        grouped = df.groupby(['county', 'is_business', 'is_consumption', 'hour', 'day'])
        df['target_rolling_allp_avg_hour_hour_day_4w'] = grouped['target'].transform(lambda x: x.rolling(window=4, min_periods=1).mean())
        
        #All of estonia
        grouped = df.groupby(['is_business', 'product_type', 'is_consumption'])
        df['target_rolling_avg_24h_estonia'] = grouped['target'].transform(lambda x: x.rolling(window=24, min_periods=1).mean())

        grouped = df.groupby(['is_business', 'product_type', 'is_consumption', 'hour'])
        df['target_rolling_avg_hour_7d_estonia'] = grouped['target'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

        # grouped = df.groupby(['is_business', 'product_type', 'is_consumption', 'hour', 'day'])
        # df['target_rolling_avg_hour_hour_day_4w_estonia'] = grouped['target'].transform(lambda x: x.rolling(window=4, min_periods=1).mean())

        grouped = df.groupby(['is_business', 'is_consumption'])
        df['target_rolling_allp_avg_24h_estonia'] = grouped['target'].transform(lambda x: x.rolling(window=24, min_periods=1).mean())

        grouped = df.groupby(['is_business', 'is_consumption', 'hour'])
        df['target_rolling_allp_avg_hour_7d_estonia'] = grouped['target'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

        # grouped = df.groupby(['is_business', 'is_consumption', 'hour', 'day'])
        # df['target_rolling_allp_avg_hour_hour_day_4w_estonia'] = grouped['target'].transform(lambda x: x.rolling(window=4, min_periods=1).mean())
        
        df = df.drop(['hour', 'day'], axis=1)

        if df.target.isna().sum() > 0:
            print("### NAS IN TARGET FEATURES")
            # display(df)

        return df
    
    def init_revealed_targets(self, df):
        
        df['datetime'] = pd.to_datetime(df.datetime)
        df['datetime'] = df['datetime'] + dt.timedelta(days=2)
        df = self.add_revealed_target_features(df)
        return df
    
    def init_client(self, df):
        ## LAG: 2 days
        ## Add 2 days to date, join on date
        df['date'] = pd.to_datetime(df.date).dt.date
        df['date'] = df['date'] + dt.timedelta(days=2)
        # df = self.get_data_block_id(df, 'date')
        return df

    def init_weather_mapping(self):
        # https://www.kaggle.com/code/tsunotsuno/enefit-eda-baseline/notebook#Baseline
        county_point_map = {
            0: (59.4, 24.7), # "HARJUMAA"
            1 : (58.8, 22.7), # "HIIUMAA"
            2 : (59.1, 27.2), # "IDA-VIRUMAA"
            3 : (58.8, 25.7), # "JÄRVAMAA"
            4 : (58.8, 26.2), # "JÕGEVAMAA"
            5 : (59.1, 23.7), # "LÄÄNE-VIRUMAA"
            6 : (59.1, 23.7), # "LÄÄNEMAA"
            7 : (58.5, 24.7), # "PÄRNUMAA"
            8 : (58.2, 27.2), # "PÕLVAMAA"
            9 : (58.8, 24.7), # "RAPLAMAA"
            10 : (58.5, 22.7),# "SAAREMAA"
            11 : (58.5, 26.7),# "TARTUMAA"
            12 : (58.5, 25.2),# "UNKNOWNN" (center of the map)
            13 : (57.9, 26.2),# "VALGAMAA"
            14 : (58.2, 25.7),# "VILJANDIMAA"
            15 : (57.9, 27.2) # "VÕRUMAA"
        }
        # Convert the dictionary to a list of tuples
        data = [(county_code, lat, lon) for county_code, (lat, lon) in county_point_map.items()]

        # Create DataFrame
        df = pd.DataFrame(data, columns=['county', 'latitude', 'longitude'])
        
        return df
    
    def add_date_features(self, df):
        df['year'] = df['datetime'].dt.year
        df['month'] = df['datetime'].dt.month
        df['day'] = df['datetime'].dt.day
        df['hour'] = df['datetime'].dt.hour
        df['quarter'] = df['datetime'].dt.quarter
        df['day_of_week'] = df['datetime'].dt.day_of_week
        df['day_of_year'] = df['datetime'].dt.dayofyear
        df['week_of_year'] = df['datetime'].dt.isocalendar().week
        df['is_weekend'] = df['datetime'].dt.day_of_week >= 5
        df['is_month_start'] = df['datetime'].dt.is_month_start
        df['is_month_end'] = df['datetime'].dt.is_month_end
        df['is_quarter_start'] = df['datetime'].dt.is_quarter_start
        df['is_quarter_end'] = df['datetime'].dt.is_quarter_end
        df['is_year_start'] = df['datetime'].dt.is_year_start
        df['is_year_end'] = df['datetime'].dt.is_year_end
        df['season'] = df['datetime'].dt.month % 12 // 3 + 1
        df['hour_sin'] = np.sin(df['datetime'].dt.hour * (2. * np.pi / 24))
        df['hour_cos'] = np.cos(df['datetime'].dt.hour * (2. * np.pi / 24))
        # Calculate sin and cos for day of year
        days_in_year = 365.25  # accounts for leap year
        df['day_of_year_sin'] = np.sin((df['day_of_year'] - 1) * (2 * np.pi / days_in_year))
        df['day_of_year_cos'] = np.cos((df['day_of_year'] - 1) * (2 * np.pi / days_in_year))
        return df
    
    def add_ee_holidays(self, df):
        import holidays
        # Define Estonia public holidays
        ee_holidays = holidays.CountryHoliday('EE')
        
        print(df['date'].isna().sum())
        
        def find_problem(x):
            try:
                return x in ee_holidays
            except Exception as e:
                print(x)
                raise e

        # Function to check if the date is a holiday
        df['is_ee_holiday'] = df['date'].apply(lambda x: x in ee_holidays)

        return df
    
    def create_log_cols(self, df):
        log_cols = ['target_lag_1h', 'target_lag_2h', 'target_lag_3h', 'target_lag_4h',
       'target_lag_5h', 'target_lag_6h', 'target_lag_7h', 'target_lag_8h',
       'target_lag_9h', 'target_lag_10h', 'target_lag_11h', 'target_lag_12h',
       'target_lag_13h', 'target_lag_14h', 'target_lag_15h', 'target_lag_16h',
       'target_lag_17h', 'target_lag_18h', 'target_lag_19h', 'target_lag_20h',
       'target_lag_21h', 'target_lag_22h', 'target_lag_23h', 'target_lag_24h',
       'target_lag_48h', 'target_lag_72h', 'target_lag_96h', 'target_lag_120h',
       'target_lag_144h', 'target_lag_168h', 'target_lag_264h',
       'target_lag_288h', 'eic_count', 'installed_capacity', 'temperature', 'dewpoint', 'rain',
       'snowfall', 'surface_pressure', 'cloudcover_total', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_high', 'windspeed_10m',
       'winddirection_10m', 'shortwave_radiation', 'direct_solar_radiation',
       'diffuse_radiation', 'temperature_hw_means', 'dewpoint_hw_means',
       'rain_hw_means', 'snowfall_hw_means', 'surface_pressure_hw_means',
       'cloudcover_total_hw_means', 'cloudcover_low_hw_means',
       'cloudcover_mid_hw_means', 'cloudcover_high_hw_means',
       'windspeed_10m_hw_means', 'winddirection_10m_hw_means',
       'shortwave_radiation_hw_means', 'direct_solar_radiation_hw_means',
       'diffuse_radiation_hw_means', 'temperature_hw_variances',
       'dewpoint_hw_variances', 'rain_hw_variances', 'snowfall_hw_variances',
       'surface_pressure_hw_variances', 'cloudcover_total_hw_variances',
       'cloudcover_low_hw_variances', 'cloudcover_mid_hw_variances',
       'cloudcover_high_hw_variances', 'windspeed_10m_hw_variances',
       'winddirection_10m_hw_variances', 'shortwave_radiation_hw_variances',
       'direct_solar_radiation_hw_variances', 'diffuse_radiation_hw_variances',
       'temperature_hw_lagged', 'dewpoint_hw_lagged', 'rain_hw_lagged',
       'snowfall_hw_lagged', 'surface_pressure_hw_lagged',
       'cloudcover_total_hw_lagged', 'cloudcover_low_hw_lagged', 'cloudcover_mid_hw_lagged',
       'cloudcover_high_hw_lagged', 'windspeed_10m_hw_lagged',
       'winddirection_10m_hw_lagged', 'shortwave_radiation_hw_lagged',
       'direct_solar_radiation_hw_lagged', 'diffuse_radiation_hw_lagged',
       'temperature_hw_means_hw_lagged', 'dewpoint_hw_means_hw_lagged',
       'rain_hw_means_hw_lagged', 'snowfall_hw_means_hw_lagged',
       'surface_pressure_hw_means_hw_lagged',
       'cloudcover_total_hw_means_hw_lagged',
       'cloudcover_low_hw_means_hw_lagged',
       'cloudcover_mid_hw_means_hw_lagged',
       'cloudcover_high_hw_means_hw_lagged',
       'windspeed_10m_hw_means_hw_lagged',
       'winddirection_10m_hw_means_hw_lagged',
       'shortwave_radiation_hw_means_hw_lagged',
       'direct_solar_radiation_hw_means_hw_lagged',
       'diffuse_radiation_hw_means_hw_lagged',
       'temperature_hw_variances_hw_lagged', 'dewpoint_hw_variances_hw_lagged',
       'rain_hw_variances_hw_lagged', 'snowfall_hw_variances_hw_lagged',
       'surface_pressure_hw_variances_hw_lagged',
       'cloudcover_total_hw_variances_hw_lagged',
       'cloudcover_low_hw_variances_hw_lagged',
       'cloudcover_mid_hw_variances_hw_lagged',
       'cloudcover_high_hw_variances_hw_lagged',
       'windspeed_10m_hw_variances_hw_lagged',
       'winddirection_10m_hw_variances_hw_lagged',
       'shortwave_radiation_hw_variances_hw_lagged',
       'direct_solar_radiation_hw_variances_hw_lagged',
       'diffuse_radiation_hw_variances_hw_lagged', 'temperature_fw', 'dewpoint_fw', 'cloudcover_high_fw',
       'cloudcover_low_fw', 'cloudcover_mid_fw', 'cloudcover_total_fw',
       '10_metre_u_wind_component', '10_metre_v_wind_component',
       'direct_solar_radiation_fw', 'surface_solar_radiation_downwards',
       'snowfall_fw', 'total_precipitation', 'euros_per_mwh', 'mean_euros_per_mwh_last_week',
       'mean_euros_per_mwh_same_hour_last_week', 'yesterdays_euros_per_mwh',
       'euros_per_mwh_24h_average_price', 'lowest_price_per_mwh',
       'highest_price_per_mwh', 'lowest_price_3d_avg', 'highest_price_3d_avg',
       'lowest_price_7d_avg', 'highest_price_7d_avg', 'lowest_price_14d_avg',
       'highest_price_14d_avg']
        
        log_cols = [col for col in log_cols if col in df.columns]
        
        dff = np.log1p(df[log_cols] )
        dff.rename(columns={col: col + "_log" for col in log_cols}, inplace=True)
        return pd.concat([df, dff], axis=1)
        
    
    def remove_cols(self, df):
        col_list = [
                   'prediction_unit_id',
                    'date_train',
                    'hour_part',
                   'date_client',
                    'forecast_date_elec_price',
                    'origin_date_elec_price',
                    'forecast_date_gas_price',
                    'origin_date_gas_price',
                    'datetime_hist_weath',
                   'hour_part_hist_weath_latest',
                    'datetime_hist_weath_latest',
                   'origin_datetime',
                   'hour_part_fore_weath',
                    'id',
                     'data_block_id',
                     'prediction_unit_id',
                     'date',
                    'data_block_id_rt',
                     'row_id_rt',
                     'prediction_unit_id_rt',
                    'data_block_id_client',
                    'latitude',
                     'longitude',
                     'data_block_id_hw',
                    'start_time',
                     'end_time',
                     'time_code',
                     'group',
                    'data_block_id_hw_means',
                    'data_block_id_hw_variances',
                     'location_id',
                     'date_hw',
                     'datetime_hw_lagged',
                    'latitude_hw_lagged',
                     'longitude_hw_lagged',
                     'data_block_id_hw_lagged',
                     'start_time_hw_lagged',
                     'end_time_hw_lagged',
                     'time_code_hw_lagged',
                     'group_hw_lagged',
                    'data_block_id_hw_means_hw_lagged',
                    'data_block_id_hw_variances_hw_lagged',
                    'location_id_hw_lagged',
                     'latitude_fw',
                     'longitude_fw',
                     'origin_datetime',
                    'data_block_id_fw',
                     'forecast_datetime',
                    'data_block_id_elec',
                    'forecast_date',
                    'origin_date',
                     'data_block_id_gasp',
                   ]
        columns_to_drop = [col for col in col_list if col in df.columns]
        df = df.drop(columns_to_drop, axis=1)
        return df
    
    def remove_test_cols(self, df):
        col_list = [
                   'prediction_unit_id',
                    'date_train',
                    'hour_part',
                   'date_client',
                    'forecast_date_elec_price',
                    'origin_date_elec_price',
                    'forecast_date_gas_price',
                    'origin_date_gas_price',
                    'datetime_hist_weath',
                   'hour_part_hist_weath_latest',
                    'datetime_hist_weath_latest',
                   'origin_datetime',
                   'hour_part_fore_weath',
                     'data_block_id',
                     'prediction_unit_id',
                     'date',
                    'data_block_id_rt',
                     'row_id_rt',
                     'prediction_unit_id_rt',
                    'data_block_id_client',
                    'latitude',
                     'longitude',
                     'data_block_id_hw',
                    'start_time',
                     'end_time',
                     'time_code',
                     'group',
                    'data_block_id_hw_means',
                    'data_block_id_hw_variances',
                     'location_id',
                     'date_hw',
                     'datetime_hw_lagged',
                    'latitude_hw_lagged',
                     'longitude_hw_lagged',
                     'data_block_id_hw_lagged',
                     'start_time_hw_lagged',
                     'end_time_hw_lagged',
                     'time_code_hw_lagged',
                     'group_hw_lagged',
                    'data_block_id_hw_means_hw_lagged',
                    'data_block_id_hw_variances_hw_lagged',
                    'location_id_hw_lagged',
                     'latitude_fw',
                     'longitude_fw',
                     'origin_datetime',
                    'data_block_id_fw',
                     'forecast_datetime',
                    'data_block_id_elec',
                    'forecast_date',
                    'origin_date',
                     'data_block_id_gasp',
                    'id'
                   ]
        columns_to_drop = [col for col in col_list if col in df.columns]
        df = df.drop(columns_to_drop, axis=1)
        return df
    
    def join_data(self, train, revealed_targets, client, historical_weather, forecast_weather, electricity_prices, gas_prices):
        df = train
        display("TRAIN")
        display(df)
        display("revealed_targets")
        display(revealed_targets.sort_values(['datetime', 'row_id']))
        df = df.merge(revealed_targets, how='left', on=('datetime', 'county', 'is_business', 'product_type', 'is_consumption'), suffixes=('', '_rt'))
        display("MERGED with revealed targets")
        display(df.sort_values(['datetime', 'row_id']))
        df = df.merge(client, how='left', on=('date', 'county', 'is_business', 'product_type'), suffixes=('', '_client'))
        df = df.merge(historical_weather, how='left', on=('datetime', 'county'), suffixes=('', '_hw'))
        df = df.merge(forecast_weather, how='left', on=('datetime', 'county'), suffixes=('', '_fw'))
        df = df.merge(electricity_prices, how='left', on='datetime', suffixes=('', '_elec'))
        df['date'] = pd.to_datetime(df['date'])
        df = df.merge(gas_prices, how='left', on='date', suffixes=('', '_gasp'))
        df = self.add_date_features(df)
        df = self.add_ee_holidays(df)
        return df
    
    def add_test_data(self, test, revealed_targets, client, historical_weather,
            forecast_weather, electricity_prices, gas_prices):
        dfs = [test.copy(), revealed_targets, client, historical_weather,
                 forecast_weather, electricity_prices, gas_prices]
        for i, df in enumerate(dfs):
            if 'datetime' in df.columns:
                df['datetime'] = pd.to_datetime(df.datetime)
                col = 'datetime'
            if 'prediction_datetime' in df.columns:
                df['datetime'] = pd.to_datetime(df.prediction_datetime)
                col = 'datetime'
                df = df.drop('prediction_datetime', axis=1)
            if 'forecast_date' in df.columns:
                df['forecast_date'] = pd.to_datetime(df['forecast_date'])
                col = 'forecast_date'
            if 'forecast_datetime' in df.columns:
                df['forecast_datetime'] = pd.to_datetime(df['forecast_datetime'])
                col = 'forecast_datetime'
                
            self.test_orig_dfs[i] = pd.concat([ self.test_orig_dfs[i], df ])

    def update_targets(self, df, revealed_targets):
        # Select only the necessary columns from revealed_targets for merging
        cols_to_merge = ['county', 'is_business', 'product_type', 'is_consumption', 'datetime', 'target']

        # Performing a left join with an indicator
        merged_df = df.merge(revealed_targets[cols_to_merge], 
                            on=['county', 'is_business', 'product_type', 'is_consumption', 'datetime'], 
                            how='left',
                            suffixes=('', '_revealed'),
                            indicator=True)

        # Update 'target' in df only where it's NaN and the row is from the left DataFrame only
        mask = (merged_df['_merge'] == 'left_only') & (merged_df['target'].isna())
        df.loc[df.index[mask], 'target'] = merged_df.loc[mask, 'target_revealed']

        return df

    def add_to_df(self, df, revealed_targets):
        if 'currently_scored' in df.columns:
            df = df.drop('currently_scored', axis=1)
        # Filter out rows in df that have indices already in self.df
        o_index = self.df.index
        n_index = df.index
        df = df[~df.index.isin(self.df.index)]
        # Add to training data only if df is not empty after filtering
        if not df.empty:
            self.df = pd.concat([self.df, df])
        else:
            print('df was empty')
            display(o_index)
            display(n_index)
        # Add revealed targets if appropriate to target column
        self.df = self.update_targets(self.df, revealed_targets)
    
    def process_test_data_timestep(self, test, revealed_targets, client, historical_weather,
            forecast_weather, electricity_prices, gas_prices):
        test_revealed_targets = revealed_targets
        #append test data to test data cache
        marker = time.time()
        self.add_test_data(test, revealed_targets, client, historical_weather,
            forecast_weather, electricity_prices, gas_prices)
        diff = time.time()-marker
        print(f"Time to add test: {diff}")
        
        # process test data
        marker = time.time()
        test = self.init_train(self.test_orig_dfs[0])
        revealed_targets = self.init_revealed_targets(self.test_orig_dfs[1])
        client = self.init_client(self.test_orig_dfs[2])
        historical_weather = self.init_historical_weather(self.test_orig_dfs[3])
        forecast_weather = self.init_forecast_weather(self.test_orig_dfs[4])
        electricity_prices = self.init_electricity(self.test_orig_dfs[5])
        gas_prices = self.init_gas_prices(self.test_orig_dfs[6])
        df_all_cols = self.join_data(test, revealed_targets, client, historical_weather,
            forecast_weather, electricity_prices, gas_prices)
        if self.add_log_cols:
            df_all_cols = self.create_log_cols(df_all_cols)
        df = self.remove_test_cols(df_all_cols)
        diff = time.time()-marker
        print(f"Time to process test data: {diff}")
        
        #reindex to get row_id as the index
        # display(df)
        df = df.drop_duplicates('row_id')
        df.index = df.row_id.tolist()

        marker = time.time()
        self.add_to_df(df, test_revealed_targets)
        diff = time.time()-marker
        print(f"Time to add test to df: {diff}")

        #trim original dfs to prevent processing time taking too long
        self.trim_test_orig_dfs(self.test_orig_dfs)

        return df
        

In [67]:
%%time
datablock = 274

train = pd.read_csv('data/train.csv')
train = train[train.data_block_id < datablock]
display(train)

revealed_targets = pd.read_csv('data/train.csv')
revealed_targets = revealed_targets[revealed_targets.data_block_id < datablock-2].copy()
revealed_targets['data_block_id'] += 2
display(revealed_targets)


client = pd.read_csv('data/client.csv')
client= client[client.data_block_id < datablock]

historical_weather = pd.read_csv('data/historical_weather.csv')
historical_weather = historical_weather[historical_weather.data_block_id < datablock]

forecast_weather = pd.read_csv('data/forecast_weather.csv')
forecast_weather = forecast_weather[forecast_weather.data_block_id < datablock]

electricity_prices = pd.read_csv('data/electricity_prices.csv')
electricity_prices = electricity_prices[electricity_prices.data_block_id < datablock]

gas_prices = pd.read_csv('data/gas_prices.csv')
gas_prices = gas_prices[gas_prices.data_block_id < datablock]

data_processor = TrainDataProcessor(train, revealed_targets, client, historical_weather, forecast_weather, electricity_prices, gas_prices, add_log_cols=False)

data_processor.df

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2
...,...,...,...,...,...,...,...,...,...
851227,15,1,0,114.072,1,2022-06-01 23:00:00,273,851227,64
851228,15,1,1,0.000,0,2022-06-01 23:00:00,273,851228,59
851229,15,1,1,36.401,1,2022-06-01 23:00:00,273,851229,59
851230,15,1,3,0.000,0,2022-06-01 23:00:00,273,851230,60


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,2,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,2,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,2,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,2,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,2,4,2
...,...,...,...,...,...,...,...,...,...
844795,15,1,0,108.908,1,2022-05-30 23:00:00,273,844795,64
844796,15,1,1,0.000,0,2022-05-30 23:00:00,273,844796,59
844797,15,1,1,40.896,1,2022-05-30 23:00:00,273,844797,59
844798,15,1,3,1.000,0,2022-05-30 23:00:00,273,844798,60


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021-09-01
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021-09-01
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021-09-01
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021-09-01
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021-09-01
...,...,...,...,...,...,...,...,...,...,...
851227,15,1,0,114.072,1,2022-06-01 23:00:00,273,851227,64,2022-06-01
851228,15,1,1,0.000,0,2022-06-01 23:00:00,273,851228,59,2022-06-01
851229,15,1,1,36.401,1,2022-06-01 23:00:00,273,851229,59,2022-06-01
851230,15,1,3,0.000,0,2022-06-01 23:00:00,273,851230,60,2022-06-01


'df'

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,hour,day,id
datetime,,,,,,,,,,,
2021-09-03 00:00:00,0,0,1,0.713,0,2,0,0,0,4,0_0_1_0
2021-09-03 00:00:00,11,0,2,7.620,1,2,89,44,0,4,11_0_2_1
2021-09-03 00:00:00,11,0,2,0.000,0,2,88,44,0,4,11_0_2_0
2021-09-03 00:00:00,11,0,1,21.099,1,2,87,43,0,4,11_0_1_1
2021-09-03 00:00:00,11,0,1,0.000,0,2,86,43,0,4,11_0_1_0
...,...,...,...,...,...,...,...,...,...,...,...
2022-06-01 23:00:00,4,0,1,0.001,0,273,844702,15,23,2,4_0_1_0
2022-06-01 23:00:00,3,1,3,739.677,1,273,844701,14,23,2,3_1_3_1
2022-06-01 23:00:00,3,1,3,0.000,0,273,844700,14,23,2,3_1_3_0


...

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,id,target_lag_1h,...,target_lag_288h,target_rolling_avg_24h,target_rolling_avg_hour_7d,target_rolling_allp_avg_24h,target_rolling_allp_avg_hour_7d,target_rolling_allp_avg_hour_hour_day_4w,target_rolling_avg_24h_estonia,target_rolling_avg_hour_7d_estonia,target_rolling_allp_avg_24h_estonia,target_rolling_allp_avg_hour_7d_estonia
datetime,,,,,,,,,,,,,,,,,,,,,
2021-09-03 00:00:00,0,0,1,0.713,0,2,0,0,0_0_1_0,NaN,...,NaN,0.713000,0.713000,0.713000,0.713000,0.713000,0.713000,0.713000,0.713000,0.713000
2021-09-03 00:00:00,0,0,1,96.590,1,2,1,0,0_0_1_1,NaN,...,NaN,96.590000,96.590000,256.921000,256.921000,256.921000,19.134100,21.204857,68.124565,119.252571
2021-09-03 00:00:00,0,0,2,0.000,0,2,2,1,0_0_2_0,NaN,...,NaN,0.000000,0.000000,1.205667,1.205667,1.205667,0.000000,0.000000,0.165130,0.427429
2021-09-03 00:00:00,0,0,2,17.314,1,2,3,1,0_0_2_1,NaN,...,NaN,17.314000,17.314000,337.086500,337.086500,337.086500,12.467000,12.467000,66.830682,111.059857
2021-09-03 00:00:00,0,0,3,2.904,0,2,4,2,0_0_3_0,NaN,...,NaN,2.904000,2.904000,1.808500,1.808500,1.808500,0.280455,0.428000,0.172636,0.428000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-01 23:00:00,15,1,0,108.908,1,273,844795,64,15_1_0_1,113.134,...,128.295,148.408500,116.198143,202.630292,137.274857,74.630250,471.246500,313.735571,642.526333,324.414000
2022-06-01 23:00:00,15,1,1,0.000,0,273,844796,59,15_1_1_0,0.000,...,0.000,55.732333,0.000000,31.139708,0.142857,0.000000,0.026083,0.000000,2.235750,0.569571
2022-06-01 23:00:00,15,1,1,40.896,1,273,844797,59,15_1_1_1,43.256,...,39.851,42.087583,38.239857,205.342708,127.022000,129.226750,165.953000,144.902143,674.855875,200.259000


'MERGED with revealed targets'

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date,...,target_lag_288h,target_rolling_avg_24h,target_rolling_avg_hour_7d,target_rolling_allp_avg_24h,target_rolling_allp_avg_hour_7d,target_rolling_allp_avg_hour_hour_day_4w,target_rolling_avg_24h_estonia,target_rolling_avg_hour_7d_estonia,target_rolling_allp_avg_24h_estonia,target_rolling_allp_avg_hour_7d_estonia
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851227,15,1,0,114.072,1,2022-06-01 23:00:00,273,851227,64,2022-06-01,...,128.295,148.408500,116.198143,202.630292,137.274857,74.63025,471.246500,313.735571,642.526333,324.414000
851228,15,1,1,0.000,0,2022-06-01 23:00:00,273,851228,59,2022-06-01,...,0.000,55.732333,0.000000,31.139708,0.142857,0.00000,0.026083,0.000000,2.235750,0.569571
851229,15,1,1,36.401,1,2022-06-01 23:00:00,273,851229,59,2022-06-01,...,39.851,42.087583,38.239857,205.342708,127.022000,129.22675,165.953000,144.902143,674.855875,200.259000
851230,15,1,3,0.000,0,2022-06-01 23:00:00,273,851230,60,2022-06-01,...,0.000,94.921667,0.285714,22.381875,0.285714,0.25000,0.371375,0.354000,0.282708,0.225571


0
CPU times: total: 1min 17s
Wall time: 3min 42s


,county,is_business,product_type,target,is_consumption,datetime,row_id,target_rt,target_lag_1h,target_lag_2h,...,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season,hour_sin,hour_cos,day_of_year_sin,day_of_year_cos,is_ee_holiday
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,NaN,NaN,NaN,...,False,False,False,False,4,0.000000,1.000000,-0.861693,-0.507430,False
1,0,0,1,96.590,1,2021-09-01 00:00:00,1,NaN,NaN,NaN,...,False,False,False,False,4,0.000000,1.000000,-0.861693,-0.507430,False
2,0,0,2,0.000,0,2021-09-01 00:00:00,2,NaN,NaN,NaN,...,False,False,False,False,4,0.000000,1.000000,-0.861693,-0.507430,False
3,0,0,2,17.314,1,2021-09-01 00:00:00,3,NaN,NaN,NaN,...,False,False,False,False,4,0.000000,1.000000,-0.861693,-0.507430,False
4,0,0,3,2.904,0,2021-09-01 00:00:00,4,NaN,NaN,NaN,...,False,False,False,False,4,0.000000,1.000000,-0.861693,-0.507430,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851353,15,1,0,114.072,1,2022-06-01 23:00:00,851227,108.908,113.134,127.814,...,False,False,False,False,3,-0.258819,0.965926,0.517586,-0.855631,False
851354,15,1,1,0.000,0,2022-06-01 23:00:00,851228,0.000,0.000,0.001,...,False,False,False,False,3,-0.258819,0.965926,0.517586,-0.855631,False
851355,15,1,1,36.401,1,2022-06-01 23:00:00,851229,40.896,43.256,42.361,...,False,False,False,False,3,-0.258819,0.965926,0.517586,-0.855631,False
851356,15,1,3,0.000,0,2022-06-01 23:00:00,851230,1.000,0.346,2.162,...,False,False,False,False,3,-0.258819,0.965926,0.517586,-0.855631,False


In [68]:
import pickle

with open('data_processor_test.pkl', 'wb') as f:
    pickle.dump(data_processor, f)

In [69]:
with open('data_processor_test.pkl', 'rb') as f:
    data_processor = pickle.load(f)
data_processor.df

,county,is_business,product_type,target,is_consumption,datetime,row_id,target_rt,target_lag_1h,target_lag_2h,...,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season,hour_sin,hour_cos,day_of_year_sin,day_of_year_cos,is_ee_holiday
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,NaN,NaN,NaN,...,False,False,False,False,4,0.000000,1.000000,-0.861693,-0.507430,False
1,0,0,1,96.590,1,2021-09-01 00:00:00,1,NaN,NaN,NaN,...,False,False,False,False,4,0.000000,1.000000,-0.861693,-0.507430,False
2,0,0,2,0.000,0,2021-09-01 00:00:00,2,NaN,NaN,NaN,...,False,False,False,False,4,0.000000,1.000000,-0.861693,-0.507430,False
3,0,0,2,17.314,1,2021-09-01 00:00:00,3,NaN,NaN,NaN,...,False,False,False,False,4,0.000000,1.000000,-0.861693,-0.507430,False
4,0,0,3,2.904,0,2021-09-01 00:00:00,4,NaN,NaN,NaN,...,False,False,False,False,4,0.000000,1.000000,-0.861693,-0.507430,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851353,15,1,0,114.072,1,2022-06-01 23:00:00,851227,108.908,113.134,127.814,...,False,False,False,False,3,-0.258819,0.965926,0.517586,-0.855631,False
851354,15,1,1,0.000,0,2022-06-01 23:00:00,851228,0.000,0.000,0.001,...,False,False,False,False,3,-0.258819,0.965926,0.517586,-0.855631,False
851355,15,1,1,36.401,1,2022-06-01 23:00:00,851229,40.896,43.256,42.361,...,False,False,False,False,3,-0.258819,0.965926,0.517586,-0.855631,False
851356,15,1,3,0.000,0,2022-06-01 23:00:00,851230,1.000,0.346,2.162,...,False,False,False,False,3,-0.258819,0.965926,0.517586,-0.855631,False


### Creating Test data that is a year long

In [ ]:
def create_test_df(df, name):
    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df.datetime)
        col = 'datetime'
    if 'prediction_datetime' in df.columns:
        df['prediction_datetime'] = pd.to_datetime(df.prediction_datetime)
        col = 'prediction_datetime'
    if 'forecast_date' in df.columns:
        df['forecast_date'] = pd.to_datetime(df['forecast_date'])
        col = 'forecast_date'
    if 'forecast_datetime' in df.columns:
        df['forecast_datetime'] = pd.to_datetime(df['forecast_datetime'])
        col = 'forecast_datetime'
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df.date).dt.date
        col = 'date'

    if name =='test.csv':
        df = df.drop('target', axis=1)
        df['currently_scored'] = False
        sample_submission = df[['row_id', 'data_block_id']]
        sample_submission['target'] = 0
        sample_submission = sample_submission.drop_duplicates()
        sample_submission.to_csv(f"data/example_test_files/sample_submission.csv", index=False)

    test_date = df[col].iloc[-1]  # Assuming test is a DataFrame
    start_date = test_date - pd.Timedelta(days=367)
    historical_subset = df[df[col] >= start_date]
    historical_subset = historical_subset.drop_duplicates()
    historical_subset.to_csv(f"data/example_test_files/{name}", index=False)
    

In [ ]:
%%time

import pandas as pd
import datetime as dt

train = pd.read_csv('data/train.csv')
train = train[train.data_block_id > 274].copy()

revealed_targets = pd.read_csv('data/train.csv')
revealed_targets['datetime'] = pd.to_datetime(revealed_targets.datetime)
revealed_targets = revealed_targets[revealed_targets.data_block_id > 272].copy()
revealed_targets['data_block_id'] += 2


client = pd.read_csv('data/client.csv')
client = client[client.data_block_id > 274].copy()

historical_weather = pd.read_csv('data/historical_weather.csv')
historical_weather = historical_weather[historical_weather.data_block_id > 274].copy()

forecast_weather = pd.read_csv('data/forecast_weather.csv')
forecast_weather = forecast_weather[forecast_weather.data_block_id > 274].copy()

electricity_prices = pd.read_csv('data/electricity_prices.csv')
electricity_prices = electricity_prices[electricity_prices.data_block_id > 274].copy()

gas_prices = pd.read_csv('data/gas_prices.csv')
gas_prices = gas_prices[gas_prices.data_block_id > 274].copy()

dfs = [train, revealed_targets, client, historical_weather, forecast_weather, electricity_prices, gas_prices]
display(revealed_targets)
names = ['test.csv', 'revealed_targets.csv', 'client.csv', 'historical_weather.csv', 'forecast_weather.csv', 'electricity_prices.csv', 'gas_prices.csv']

for df, name in zip(dfs, names):
    create_test_df(df, name)

# Testing

For my experimental CV, I want to take the approach of doing a stratified CV by time - splitting the year into 4 different parts, basically testing the model on each season, 3 months at a time. There was something in the kaggle forums that recommended something like this:

Key: 
= -> training data
+ -> CV data

4 splits in time:
1. =============+++
2. ================+++
3. ===================+++
4. ======================+++



The data starts on 2021-09-01 and ends on 2023-05-31

BUT we don't have enough data to do that properly. So, my CV will instead be:


(Thanks chatgpt)

Splitting the period from 2022-09-01 to 2023-05-31 into five equal parts, here are the date ranges for each segment:

#### First Segment:

From 2022-09-01 to 2022-10-24

#### Second Segment:

From 2022-10-25 to 2022-12-17

#### Third Segment:

From 2022-12-18 to 2023-02-09

#### Fourth Segment:

From 2023-02-10 to 2023-04-04

#### Fifth Segment:

From 2023-04-05 to 2023-05-29


In [70]:
def fill_drop_na(df):
    df = df[~df.target.isna()]
    df = df[~df.target_rolling_avg_24h.isna()]
    means = df.mean()
    # For each column, add an indicator column for NA values
    # for col in df.columns:
    #     if df[col].isna().any():
    #         df[f'{col}_is_na'] = df[col].isna()
    df = df.fillna(means)
    return df, means

In [71]:
%%time
processed_df_no_na, means = fill_drop_na(data_processor.df)
processed_df_no_na.isna().sum()

CPU times: total: 1.95 s
Wall time: 5.16 s


county             0
is_business        0
product_type       0
target             0
is_consumption     0
                  ..
hour_sin           0
hour_cos           0
day_of_year_sin    0
day_of_year_cos    0
is_ee_holiday      0
Length: 238, dtype: int64

In [72]:
# processed_df_no_na['target_installed_capacity'] = processed_df_no_na['target'] / processed_df_no_na['installed_capacity'] * 1000
# processed_df_no_na

In [73]:
from datetime import datetime

# cv_range = [('2022-09-10', '2023-05-31')]

# Note: This training range means that the models don't have any observations for months 6,7,8 - which isn't ideal. But we'll see. I might try another one with my previous range and compare blending results.
cv_range = [('2022-06-01', '2023-05-31')]

# Function to convert a date string into a datetime object
def to_datetime(date_str):
    return datetime.strptime(date_str, '%Y-%m-%d')

# Converting the date strings in cv_ranges to datetime objects
datetime_cv_ranges = [(to_datetime(start), to_datetime(end)) for start, end in cv_range]
datetime_cv_ranges

date_filter = data_processor.df_all_cols.date[processed_df_no_na.index]
date_filter

cv1_train = processed_df_no_na[date_filter <= datetime_cv_ranges[0][0]]
cv1_test = processed_df_no_na[(date_filter <= datetime_cv_ranges[0][1]) & (date_filter > datetime_cv_ranges[0][0])]

In [74]:
import datetime as dt
print(to_datetime('2023-04-05') + dt.timedelta(days=14))
print(to_datetime('2023-04-05') + dt.timedelta(days=48))

2023-04-19 00:00:00
2023-05-23 00:00:00


In [75]:
cv1_train[['year' ,'month', 'day']]

,year,month,day
5856,2021,9,3
5857,2021,9,3
5858,2021,9,3
5859,2021,9,3
5860,2021,9,3
...,...,...,...
851353,2022,6,1
851354,2022,6,1
851355,2022,6,1
851356,2022,6,1


In [76]:
cv1_test[['year' ,'month', 'day']]

,year,month,day


In [77]:
processed_df_no_na[['year', 'month', 'day']]

,year,month,day
5856,2021,9,3
5857,2021,9,3
5858,2021,9,3
5859,2021,9,3
5860,2021,9,3
...,...,...,...
851353,2022,6,1
851354,2022,6,1
851355,2022,6,1
851356,2022,6,1


In [57]:
grouped = processed_df_no_na.groupby(['year', 'month', 'day'])

for i, ((year, month, day), group) in enumerate(grouped):
    # Extracting the corresponding data from df_val_target2
    target_data = processed_df_no_na.loc[group.index]
    print(year, month, day)
    
    
    if i == 10:
        break

2021 9 3
2021 9 4
2021 9 5
2021 9 6
2021 9 7
2021 9 8
2021 9 9
2021 9 10
2021 9 11
2021 9 12
2021 9 13


### Retrainable Model

In [59]:
%%time

train_pred_list = []
train_mae_list = []
train_targets_list = []

pred_list = []
mae_list = []
val_targets_list = []

daily_mae_list = []
date_check_list = []
daily_preds = []
daily_true = []

df = processed_df_no_na
i=0
start = datetime_cv_ranges[i][0] + dt.timedelta(days=0*14)
stop = datetime_cv_ranges[i][0] + dt.timedelta(days=(0+1)*14)
train = processed_df_no_na[date_filter <= start]
val = processed_df_no_na[(date_filter <= stop) & (date_filter > start)]

print(f"Fold {i}, period {f}")
print(f"Train rows: {len(train)}")
print(f"Val rows: {len(val)}")

target_cols = ['target']
drop_cols = ['target', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances',
            'snowfall_fw', 'snowfall_hw_means', 'datetime', 'row_id']

df_train_target = train[target_cols]
df_train_data = train.drop(drop_cols, axis=1)

df_val_target2 = val[target_cols]
df_val_data2 = val.drop(drop_cols, axis=1)

cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
        'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
        'is_year_start', 'is_year_end', 'season'] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
cat_features = [c for c in cat_features if c in df_train_data.columns]

# We leave max_depth as -1
# Tune num_leaves, default is 31, let's double it       

params = {'lambda_l1': 0.7466999841658806, 'lambda_l2': 3.2140838539606458, 'learning_rate': 0.13753679743025782, 'max_bin': 250, 'min_data_in_leaf': 150, 'n_estimators': 5593,  
            'metric': 'mae', 'n_jobs': 22, 'boosting': 'dart', 'objective': 'tweedie', 'device':'gpu'}

clf = LGBMRegressor(**params, random_state=42, verbose=0, importance_type='gain')

clf.fit(df_train_data, df_train_target.target, categorical_feature=cat_features)

y_pred = clf.predict(df_train_data)
train_pred_list.append(y_pred)

from sklearn.metrics import mean_absolute_error

# Assuming you have two pandas Series: y_true and y_pred
mae = mean_absolute_error(df_train_target.target, y_pred)
train_mae_list.append(mae)
train_targets_list.append(df_train_target.target)
print(f" Train Mean Absolute Error_consumption:", mae)

# y_pred_val = clf.predict(df_val_data2)
# pred_list.append(y_pred_val)

# mae = mean_absolute_error(df_val_target2.target, y_pred_val)
# val_targets_list.append(df_val_target2.target)
# mae_list.append(mae)
# print("Val Mean Absolute Error:", mae)

# importance = pd.DataFrame({'importance':clf2.feature_importances_, 'name':clf2.feature_name_})
# importance = importance.sort_values('importance', ascending=False)
# display(importance.head(30))
# display(importance.tail(30))
print()
print()  

Fold 0, period <_io.BufferedReader name='data_processor_test.pkl'>
Train rows: 837658
Val rows: 0
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.7466999841658806, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7466999841658806
[LightGBM] [Warning] lambda_l2 is set=3.2140838539606458, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.2140838539606458
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] lambda_l1 is set=0.7466999841658806, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7466999841658806
[LightGBM] [Warning] lambda_l2 is set=

In [60]:
import pickle

with open('experiments/clf.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [61]:
import pickle

with open('experiments/clf.pkl', 'rb') as f:
    clf = pickle.load(f)

In [62]:
clf.set_params(**{'verbose':-1})

LGBMRegressor(boosting='dart', device='gpu', importance_type='gain',
              lambda_l1=0.7466999841658806, lambda_l2=3.2140838539606458,
              learning_rate=0.13753679743025782, max_bin=250, metric='mae',
              min_data_in_leaf=150, n_estimators=5593, n_jobs=22,
              objective='tweedie', random_state=42, verbose=-1)

In [63]:
def train_model(df):
    train = df
    print(f"Train rows: {len(train)}")

    target_cols = ['target']
    drop_cols = ['target', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances',
                'snowfall_fw', 'snowfall_hw_means', 'datetime', 'row_id', 'currently_scored']

    df_train_target = train[target_cols]
    drop_cols = [c for c in drop_cols if c in train.columns] 
    df_train_data = train.drop(drop_cols, axis=1)

    cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
            'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
            'is_year_start', 'is_year_end', 'season'] + list(df_train_data.columns[df_train_data.columns.str.contains('is_na')])
    cat_features = [c for c in cat_features if c in df_train_data.columns]

    # We leave max_depth as -1
    # Tune num_leaves, default is 31, let's double it       

    params = {'lambda_l1': 0.7466999841658806, 'lambda_l2': 3.2140838539606458, 'learning_rate': 0.13753679743025782, 'max_bin': 250, 'min_data_in_leaf': 150, 'n_estimators': 5593,  
                'metric': 'mae', 'n_jobs': 22, 'boosting': 'dart', 'objective': 'tweedie', 'device':'gpu'}

    clf = LGBMRegressor(**params, random_state=42, verbose=-1, importance_type='gain')

    clf.fit(df_train_data, df_train_target.target, categorical_feature=cat_features)

    return clf

## Testing Retraining model

#### Up to testing my data processor to see if it works

In [64]:
import warnings
warnings.filterwarnings('ignore', 'SettingWithCopyWarning')

In [78]:
%%time


with open('data_processor_test.pkl', 'rb') as f:
    data_processor = pickle.load(f)

processed_df_no_na, means = fill_drop_na(data_processor.df)


from data import public_timeseries_testing_util as enefit

env = enefit.make_env()

for i, (test, revealed_targets, client, historical_weather,
            forecast_weather, electricity_prices, gas_prices, sample_submission) in enumerate(env.iter_test()):
    test_data = data_processor.process_test_data_timestep(test, revealed_targets, client, historical_weather, forecast_weather, electricity_prices, gas_prices)
    
    print(f"Day: {i}")
    # display(test_data)
    # display(data_processor.df.columns)

    # filter based on predictions
    test_data['currently_scored'] = test_data['currently_scored'].fillna(True)
    score_mask = ~test_data.currently_scored
    test_data = test_data[score_mask]
    sample_submission = test_data[['row_id']]

    # we need to drop a few columns
    drop_cols = ['target', 'quarter', 'season', 'is_year_end', 'is_year_start', 'is_month_end', 'is_quarter_end', 'is_quarter_start', 'is_month_start', 'snowfall_hw_lagged', 'snowfall_hw_variances','snowfall_fw', 'snowfall_hw_means', 
                 'datetime', 'row_id', 'currently_scored']

    test_data = test_data.drop(drop_cols, axis=1)

    cat_features = ["county", "is_business", "product_type", "is_consumption", 'month', 'hour', 'quarter',
'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start' ,'is_quarter_end', 
'is_year_start', 'is_year_end', 'season'] + list(test_data.columns[test_data.columns.str.contains('is_na')])
    cat_features = [c for c in cat_features if c in test_data.columns]

    #retrain every 30 days for testing
    if (i!= 0) and (i%30 == 0):
        print(f"Train rows: {len(data_processor.df)}")
        clf = train_model(data_processor.df)

    preds = clf.predict(test_data)

    # display(preds)
    # display(sample_submission)
    # print(f"Length of preds: {len(preds)}")

    sample_submission['target'] = preds
    env.predict(sample_submission)

    # set test data currently scored to true
    data_processor.test_orig_dfs[0]['currently_scored'] = True
    print('Preds predicted')
    

Time to add test: 0.015670299530029297


'df'

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,hour,day,id
datetime,,,,,,,,,,,
2022-05-18 23:00:00,0,0,1,1.599,0,259.0,799642,0,23,2,0_0_1_0
2022-05-18 23:00:00,11,1,0,0.000,0,259.0,799740,67,23,2,11_1_0_0
2022-05-18 23:00:00,11,0,3,342.649,1,259.0,799739,45,23,2,11_0_3_1
2022-05-18 23:00:00,11,0,3,1.350,0,259.0,799738,45,23,2,11_0_3_0
2022-05-18 23:00:00,11,0,2,0.979,1,259.0,799737,44,23,2,11_0_2_1
...,...,...,...,...,...,...,...,...,...,...,...
2022-06-03 23:00:00,4,0,1,0.018,0,NaN,851134,15,23,4,4_0_1_0
2022-06-03 23:00:00,3,1,3,747.461,1,NaN,851133,14,23,4,3_1_3_1
2022-06-03 23:00:00,3,1,3,0.000,0,NaN,851132,14,23,4,3_1_3_0


'lagged_feature 1'

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,hour,day,id
datetime,,,,,,,,,,,
2022-05-19 00:00:00,0,0,1,1.599,0,259.0,799642,0,23,2,0_0_1_0
2022-05-19 01:00:00,0,0,1,0.326,0,260.0,799776,0,0,3,0_0_1_0
2022-05-19 02:00:00,0,0,1,1.438,0,260.0,799910,0,1,3,0_0_1_0
2022-05-19 03:00:00,0,0,1,0.347,0,260.0,800044,0,2,3,0_0_1_0
2022-05-19 04:00:00,0,0,1,1.361,0,260.0,800178,0,3,3,0_0_1_0
...,...,...,...,...,...,...,...,...,...,...,...
2022-06-03 20:00:00,9,1,3,116.610,1,NaN,850645,37,19,4,9_1_3_1
2022-06-03 21:00:00,9,1,3,123.902,1,NaN,850779,37,20,4,9_1_3_1
2022-06-03 22:00:00,9,1,3,94.437,1,NaN,850913,37,21,4,9_1_3_1


...

'lagged_feature 1'

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,hour,day,id
datetime,,,,,,,,,,,
2022-06-23 01:00:00,0,0,1,0.920,0,NaN,912336,0,0,3,0_0_1_0
2022-06-23 02:00:00,0,0,1,0.908,0,NaN,912470,0,1,3,0_0_1_0
2022-06-23 03:00:00,0,0,1,0.807,0,NaN,912604,0,2,3,0_0_1_0
2022-06-23 04:00:00,0,0,1,0.695,0,NaN,912738,0,3,3,0_0_1_0
2022-06-23 05:00:00,0,0,1,2.194,0,NaN,912872,0,4,3,0_0_1_0
...,...,...,...,...,...,...,...,...,...,...,...
2022-07-13 20:00:00,9,1,3,116.610,1,NaN,850645,37,19,2,9_1_3_1
2022-07-13 21:00:00,9,1,3,123.902,1,NaN,850779,37,20,2,9_1_3_1
2022-07-13 22:00:00,9,1,3,94.437,1,NaN,850913,37,21,2,9_1_3_1


'df 2'

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,hour,day,...,target_lag_23h,target_lag_24h,target_lag_48h,target_lag_72h,target_lag_96h,target_lag_120h,target_lag_144h,target_lag_168h,target_lag_264h,target_lag_288h
datetime,,,,,,,,,,,,,,,,,,,,,
2022-06-23 00:00:00,15,1,3,336.545,1,NaN,912469,60,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-23 00:00:00,11,1,0,0.000,0,NaN,912434,67,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-23 00:00:00,11,0,3,256.656,1,NaN,912433,45,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-23 00:00:00,11,0,3,0.674,0,NaN,912432,45,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-23 00:00:00,11,0,2,1.303,1,NaN,912431,44,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-13 23:00:00,4,0,1,9.686,1,NaN,851135,15,23,2,...,NaN,NaN,9.324,9.340,16.628,9.969,8.879,9.171,10.627,9.666
2022-07-13 23:00:00,4,0,1,0.018,0,NaN,851134,15,23,2,...,NaN,NaN,0.001,0.002,0.000,0.001,0.005,0.000,0.002,0.003
2022-07-13 23:00:00,3,1,3,747.461,1,NaN,851133,14,23,2,...,NaN,NaN,739.677,777.866,761.635,754.692,735.991,725.900,731.024,741.122


KeyboardInterrupt: 

## Major time is from processing test data. 

Takes 15s initially, and then already up to 18s after 5 days or so. If this stayed constant, would take 100 minutes to process data. Which is a lot, but is manageable I think. But it grows constantly. So I need a way to cut the time down. I can do this by truncating the test dfs so that they only store 14~18 days of data, and it never grows beyond that. I can also do this by using vaex or something similar (maybe).

In [ ]:
# display(test_data)
display( test_data.columns[~test_data.columns.isin(data_processor.df.columns)])